In [1]:
using AutomotiveDrivingModels
using AutoViz
using Reactive
using Interact

In [2]:
type HRHC <: DriverModel{NextState, IntegratedContinuous}
    action_context::IntegratedContinuous
    v_cmds # possible velocity commands
    δ_cmds # possible δ_cmds
    ΔXYθ # state changes associated with cmd = (v_command, δ_command)
    legal_ΔXYθ # reachable from current v, δ
    legal_v # reachable from current v, δ
    legal_δ # reachable from current v, δ
    v_idx_low::Int # index lowest reachable v_command
    v_idx_high::Int # index highest reachable v_command
    δ_idx_low::Int # index lowest reachable δ_command
    δ_idx_high::Int # index highest reachable δ_command
    successor_states # reachable next_states
    
    #car parameters with bicycle geometry model
    car_length::Float64 # wheel base
    car_width::Float64
    car_ID::Int
    
    # current v, current δ
    v::Float64
    δ::Float64
    curve_ind::Int
    Δs::Float64
    
    # planning horizon
    h::Int
    Δt::Float64
    
    # reachable range of V and δ within a single time step
    ΔV₊::Float64
    ΔV₋::Float64
    Δδ::Float64
    
    V_MIN::Float64
    V_MAX::Float64
    V_STEPS::Int
    δ_MAX::Float64
    δ_MIN::Float64
    δ_STEPS::Int

    # maximum deviation from center of track (if |t| > T_MAX, car is out of bounds)
    T_MAX::Float64
    
    # Action = Next State
    nextState::NextState
    
    function HRHC(car_ID::Int,roadway;
        car_length::Float64=4.8,
        car_width::Float64=1.8,
        v::Float64=0.0,
        δ::Float64=0.0,
        h::Int=10,
        Δt::Float64=1.0/24,
        ΔV₊::Float64=1.55/100,
        ΔV₋::Float64=3.05/100,
        Δδ::Float64=Float64(π)/16,
        V_MIN::Float64=0.0,
        V_MAX::Float64=100.0,
        V_STEPS::Int=101,
        δ_MAX::Float64=Float64(π)/8,
        δ_MIN::Float64=-Float64(π)/8,
        δ_STEPS::Int=16,
        )
        
        hrhc = new()
        
        hrhc.V_MIN=V_MIN
        hrhc.V_MAX=V_MAX
        hrhc.V_STEPS=V_STEPS
        hrhc.δ_MAX=δ_MAX
        hrhc.δ_MIN=δ_MIN
        hrhc.δ_STEPS=δ_STEPS
        
        hrhc.car_ID = car_ID
        hrhc.car_length=car_length
        hrhc.car_width=car_width
        hrhc.h=h
        hrhc.Δt=Δt
        hrhc.ΔV₊=ΔV₊
        hrhc.ΔV₋=ΔV₋
        hrhc.Δδ=Δδ
        
        hrhc.v_cmds, hrhc.δ_cmds, hrhc.ΔXYθ = MotionPrimitives(car_length,car_width,h,Δt,V_MIN,V_MAX,V_STEPS,δ_MAX,δ_MIN,δ_STEPS)
        QuadraticMask!(hrhc.ΔXYθ)
        
        hrhc.v=v
        hrhc.δ=δ
        hrhc.curve_ind=1
        Δs=roadway.segments[1].lanes[1].curve[2]-roadway.segments[1].lanes[1].curve[1]
        
        action = NextState(VehicleState(VecSE2(0,0,0),0.0))
        
        hrhc
    end
end

In [3]:
function observe!(hrhc::HRHC, scene::Scene, roadway::Roadway, egoid::Int, obstacleMap::ObstacleMap) # select action
    # this is where to set the next action
    state = scene.vehicles[hrhc.car_ID].state
    v = state.v # current v
    hrhc.v = v
    
    
    hrhc.δ = hrhc.δ_cmds[abs_cmd[1], abs_cmd[2]]
    hrhc.v = hrhc.v_cmds[abs_cmd[1], abs_cmd[2]]
    
    hrhc.curve_ind = next_state.posF.roadind.ind.i    
    hrhc.action = NextState(VehicleState(VecSE2(trajectory[1,:]),roadway,hrhc.v) # action
end

LoadError: LoadError: UndefVarError: ObstacleMap not defined
while loading In[3], in expression starting on line 1

In [4]:
# defines how to get actions
Base.rand(hrhc::HRHC) = hrhc.action